In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms

import matplotlib

import matplotlib.pyplot as plt

import os
import random
import math

from torch.utils.data.dataset import Dataset

INPUT_PATH = "./output/nodule_npy/"

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 20
nz = 100
n_epochs = 500
ngf = 64
ngpu = 1
print(device)

cuda:0


In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def chunks(arr, m):
    nchunk = int(math.ceil(len(arr) / float(m)))
    return [arr[i:i + nchunk] for i in range(0, len(arr), nchunk)]

def five_folder(arr, number):
    training_set = []
    test_set = []
    for j in range(len(arr)):
        if number == j:
            test_set.extend(arr[j])
        else:
            training_set.extend(arr[j])
    return training_set, test_set


class MyDataset(Dataset):
    def __init__(self, images):
        self.images = images

    def __getitem__(self, index):#返回的是tensor
        img = self.images[index]
        return img

    def __len__(self):
        return len(self.images)

In [4]:
# Generator
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu

        self.project = nn.Sequential(
          nn.Linear(nz, 1 * 5 * 5 * ngf * 4, bias=False)
        )
        self.deconv = nn.Sequential(
          # input is Z, going into a deconvolution
          # state size. BATCH_SIZE x (ngf*8) x 4 x 4 x 4
#           nn.ConvTranspose3d(ngf * 8, ngf * 4, kernel_size=4, stride=2, padding=1, bias=False),
#           nn.BatchNorm3d(ngf * 4),
#           nn.ReLU(True),
          # state size. BATCH_SIZE x (ngf*4) x 1 x 5 x 5
          nn.ConvTranspose3d(ngf * 4, ngf * 2, kernel_size=4, stride=2, padding=1, bias=False),
          nn.BatchNorm3d(ngf * 2),
          nn.ReLU(True),
          # state size. BATCH_SIZE x (ngf*2) x 2 x 10 x 10
          nn.ConvTranspose3d(ngf * 2, ngf, kernel_size=4, stride=2, padding=1, bias=False),
          nn.BatchNorm3d(ngf),
          nn.ReLU(True),
          # state size. BATCH_SIZE x (ngf) x 4 x 20 x 20
          nn.ConvTranspose3d(ngf, 1, kernel_size=4, stride=2, padding=1, bias=False),
          nn.Tanh()
          # state size. BATCH_SIZE x 1 x 8 x 40 x 40
        )

    def forward(self, input):
        x = self.project(input)
        # Conv3d的规定输入数据格式为(batch, channel, Depth, Height, Width)
#         print(x.shape)
        x = x.view(-1, ngf * 4, 1, 5, 5)
#         x = self.deconv(x)
        
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.deconv, x, range(self.ngpu))
        else:
            output = self.deconv(x)

        return output


# Discriminator
def softmax(input, dim=1):
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = F.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)), dim=-1)
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)

class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, out_channels=256, kernel_size=(2,9,9)):
        super(ConvLayer, self).__init__()

        self.conv = nn.Conv3d(in_channels=in_channels,
                                out_channels=out_channels,
                                kernel_size=kernel_size,
                                stride=1
                              )

    def forward(self, x):
        return F.relu(self.conv(x))


class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=32, kernel_size=(2,9,9), num_routes=32 * 12 * 12 * 6):
        super(PrimaryCaps, self).__init__()

        self.capsules = nn.ModuleList([
          nn.Conv3d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=(1,2,2), padding=0) 
                      for _ in range(num_capsules)])
  
    def forward(self, x):
        u = [capsule(x).view(x.size(0), -1, 1) for capsule in self.capsules]
        u = torch.cat(u, dim=-1)
        return self.squash(u)
  
    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm *  input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor



class DigitCaps(nn.Module):
    def __init__(self, num_capsules=1, num_routes=32 * 12 * 12 * 6, in_channels=8, out_channels=16, num_iterations=3):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules
        self.num_iterations = num_iterations
        self.route_weights = nn.Parameter(torch.randn(num_capsules, num_routes, in_channels, out_channels)).to(device)

    def forward(self, x):
        # 矩阵相乘
        # x.size(): [1, batch_size, in_capsules, 1, dim_in_capsule]
        # weight.size(): [num_capsules, 1, num_route, in_channels, out_channels]
        priors = x[None, :, :, None, :] @ self.route_weights[:, None, :, :, :]
        logits = Variable(torch.zeros(*priors.size())).to(device)

        for i in range(self.num_iterations):
            probs = softmax(logits, dim=2)
            outputs = self.squash((probs * priors).sum(dim=2, keepdim=True))

            if i != self.num_routes - 1:
                delta_logits = (priors * outputs).sum(dim=-1, keepdim=True)
                logits = logits + delta_logits
        
        return outputs.squeeze()
  
    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm * input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
    
        self.reconstraction_layers = nn.Sequential(
          nn.Linear(16 * 10, 512),
          nn.ReLU(inplace=True),
          nn.Linear(512, 1024),
          nn.ReLU(inplace=True),
          nn.Linear(1024, 784),
          nn.Sigmoid()
        )
      
    def forward(self, x, data):
        classes = (x ** 2).sum(dim=-1) ** 0.5
        classes = F.softmax(classes, dim=-1)

        _, max_length_indices = classes.max(dim=1)
        masked = Variable(torch.sparse.torch.eye(10))
        if USE_CUDA:
              masked = masked.cuda()
        masked = masked.index_select(dim=0, index=max_length_indices.squeeze(1).data)
    
        reconstructions = self.reconstraction_layers((x * masked[:, :, None, None]).view(x.size(0), -1))
        reconstructions = reconstructions.view(-1, 1, 28, 28)

        return reconstructions, masked


class CapsNet(nn.Module):
    def __init__(self, ngpu):
        super(CapsNet, self).__init__()
        self.ngpu = ngpu
        
        self.conv_layer = ConvLayer()
        self.primary_capsules = PrimaryCaps()
        self.digit_capsules = DigitCaps()

        # self.decoder = Decoder()
        # self.mse_loss = nn.MSELoss()
      
    def forward(self, data):
        if data.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.conv_layer, data, range(self.ngpu))
            output = nn.parallel.data_parallel(self.primary_capsules, output, range(self.ngpu))
            output = nn.parallel.data_parallel(self.digit_capsules, output, range(self.ngpu))
        else:
            output = self.digit_capsules(self.primary_capsules(self.conv_layer(data)))

        classes = (output ** 2).sum(dim=-1) ** 0.5        
        return classes, output

        #reconstructions, masked = self.decoder(output, data)
        #return output, reconstructions, masked

    # def loss(self, data, x, target, reconstructions):
      #   return self.margin_loss(x, target) + self.reconstruction_loss(data, reconstructions)

    # def margin_loss(self, x, labels, size_average=True):
      #   batch_size = x.size(0)

      #   v_c = torch.sqrt((x**2).sum(dim=2, keepdim=True))

      #   left = F.relu(0.9 - v_c).view(batch_size, -1)
      #   right = F.relu(v_c - 0.1).view(batch_size, -1)

      #   loss = labels * left + 0.5 * (1.0 - labels) * right
      #   loss = loss.sum(dim=1).mean()

      #   return loss

    # def reconstruction_loss(self, data, reconstructions):
      #   loss = self.mse_loss(reconstructions.view(reconstructions.size(0), -1), data.view(reconstructions.size(0), -1))
      #   return loss * 0.0005


class CapsuleLoss(nn.Module):
    def __init__(self):
        super(CapsuleLoss, self).__init__()

    def forward(self, classes, labels):
        left = F.relu(0.9 - classes, inplace=True) ** 2
        right = F.relu(classes - 0.1, inplace=True) ** 2

        margin_loss = labels * left + 0.5 * (1. - labels) * right
        margin_loss = margin_loss.sum()

        return margin_loss / classes.size(0)

In [5]:
accuracy_list = []

def train(cross_part, train_loader, single_test_set):
    netD = CapsNet(ngpu).to(device)
    criterion = CapsuleLoss()

    netG = Generator(ngpu).to(device)
    netG.apply(weights_init)

    fixed_noise = torch.randn(BATCH_SIZE, nz, device=device)
    real_label = 1
    fake_label = 0

    optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    test_loader = torch.utils.data.DataLoader(MyDataset(single_test_set), batch_size=BATCH_SIZE, shuffle=True)
    
    
    for epoch in range(n_epochs):
        print("-----THE [{}/{}] epoch start-----".format(epoch + 1, n_epochs))
        
        for j, data in enumerate(train_loader, 0):
#                     print(j, data.shape) #torch.Size([32, 1, 8, 40, 40])

#                     plt.figure()
#                     for lenc in range(data.shape[0]):
#                         for len_img in range(8):
#                             plt.subplot(2, 4, len_img + 1)
#                             pixel_array = data[lenc][0][len_img]
#                             print(pixel_array)
#                             plt.imshow(pixel_array, cmap="gray")
#                         plt.show()


#                     pixel_array = data[0][0][0][20]
#                     print(pixel_array)

#                     data = data / 255.0
#                     pixel_array = data[0][0][0][20]
#                     print(pixel_array)
            ############################
            # (1) Update D network: minimize Lm(D(x), T = 1) + Lm(D(G(z)), T = 0)
            ###########################
            # train with real
            
            netD.train()
            
            netD.zero_grad()
            real_cpu = data.to(device, dtype=torch.float)
            batch_size = real_cpu.size(0)
            label = torch.full((batch_size,), real_label, device=device)   

            classes_d_real, output_d_real = netD(real_cpu)
#                     print("ERR_D_REAL:",classes_d_real)
            errD_real = criterion(classes_d_real, label)
            errD_real.backward()
            D_x = output_d_real.mean().item()

            # train with fake
            noise = torch.randn(batch_size, nz, device=device)
            fake = netG(noise)
            label.fill_(fake_label)
            classes_d_fake, output_d_fake = netD(fake.detach())

#                     print("ERR_D_FAKE:",classes_d_fake)
            errD_fake = criterion(classes_d_fake, label)
            errD_fake.backward()
            D_G_z1 = output_d_fake.mean().item()
            errD = errD_real + errD_fake
            optimizerD.step()

            ############################
            # (2) Update G network: minimize Lm(D(G(z)), T=1)
            ###########################
            netG.zero_grad()
            label.fill_(real_label)
            netD.eval()

            classes_g, output_g = netD(fake)
#                     print("ERRG:",classes_g)

            if epoch % 5 == 0:
                print(classes_g)
            errG = criterion(classes_g, label)
            errG.backward()
            D_G_z2 = output_g.mean().item()
            optimizerG.step()

#                     print("The batch data shape is {}".format(data.shape))

            print('[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                  % (j + 1, len(train_loader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))


        print("-----THE [{}/{}] epoch end-----".format(epoch + 1, n_epochs))
        
       
    
    return test_sum
    

In [6]:
input_data = os.listdir(INPUT_PATH)
flag = 1

for radiologist in input_data: #遍历patient文件夹——study指代每一个study文件夹
    if flag != 1:
        break
    flag += 1
    if not radiologist.startswith('.'): #忽略.DS文件        
        npy_file_path = os.path.join(INPUT_PATH, radiologist)
        npy_files = os.listdir(npy_file_path)
        npy_list = []
        for i in npy_files:
            npy_path = os.path.join(npy_file_path, i)
            single_npy = np.load(npy_path)
            single_fliplr_npy = np.fliplr(single_npy)
#             single_npy = (single_npy - np.average(single_npy)) / np.std(single_npy)
#             single_fliplr_npy = (single_fliplr_npy - np.average(single_fliplr_npy)) / np.std(single_fliplr_npy)
            single_npy = (single_npy - 127.5) / 127.5
            single_fliplr_npy = (single_fliplr_npy - 127.5) / 127.5
            #print(single_npy.shape)
            npy_list.append(single_fliplr_npy)
            npy_list.append(single_npy)

#         print(npy_file_path, np.array(npy_list).shape)
        random.shuffle(npy_list)
        npy_chunks = chunks(npy_list, 5)

#         print("THIS IS the data from {}. AND NOW the 5-folder cross-valiation start:".format(npy_file_path))   
   
#         for i in range(5):
            #training
#             print("-------------THE {} part(as the test set) cross-valiation start----------------------".format(i + 1))
        
        print("THIS IS the data from {}. AND NOW the training STARTS:".format(npy_file_path))
        single_training_set, single_test_set = five_folder(npy_chunks, 4)
        train_loader = torch.utils.data.DataLoader(MyDataset(single_training_set), batch_size=BATCH_SIZE, shuffle=True)

        #print(len(train_loader), type(train_loader))
        single_test_acc = train(i, train_loader,single_test_set)


THIS IS the data from ./output/nodule_npy/Radiologist_1. AND NOW the training STARTS:
-----THE [1/500] epoch start-----
tensor([0.0032, 0.0031, 0.0030, 0.0033, 0.0028, 0.0034, 0.0035, 0.0035, 0.0036,
        0.0031, 0.0036, 0.0037, 0.0033, 0.0032, 0.0032, 0.0031, 0.0033, 0.0033,
        0.0033, 0.0031], device='cuda:0', grad_fn=<PowBackward0>)
[1/32] Loss_D: 0.8100 Loss_G: 0.8041 D(x): 0.0000 D(G(z)): -0.0000 / 0.0002
tensor([0.0084, 0.0086, 0.0081, 0.0088, 0.0090, 0.0094, 0.0089, 0.0085, 0.0079,
        0.0083, 0.0090, 0.0091, 0.0082, 0.0084, 0.0083, 0.0090, 0.0080, 0.0083,
        0.0086, 0.0084], device='cuda:0', grad_fn=<PowBackward0>)
[2/32] Loss_D: 0.7885 Loss_G: 0.7947 D(x): 0.0008 D(G(z)): 0.0002 / 0.0006
tensor([0.0112, 0.0117, 0.0113, 0.0119, 0.0118, 0.0109, 0.0111, 0.0110, 0.0110,
        0.0111, 0.0114, 0.0114, 0.0112, 0.0113, 0.0114, 0.0108, 0.0120, 0.0116,
        0.0106, 0.0110], device='cuda:0', grad_fn=<PowBackward0>)
[3/32] Loss_D: 0.7714 Loss_G: 0.7898 D(x): 0.0016 D

tensor([0.1118, 0.1329, 0.0121, 0.1182, 0.1802, 0.1453, 0.0854, 0.0423, 0.0570,
        0.1619, 0.1274, 0.1273, 0.0998, 0.1546, 0.0695, 0.0604, 0.0389, 0.1512,
        0.0447, 0.0448], device='cuda:0', grad_fn=<PowBackward0>)
[28/32] Loss_D: 0.6260 Loss_G: 0.6451 D(x): 0.0068 D(G(z)): 0.0074 / 0.0063
tensor([0.1358, 0.1217, 0.1009, 0.0457, 0.1044, 0.2136, 0.1728, 0.1402, 0.1734,
        0.1495, 0.1410, 0.1364, 0.1070, 0.1896, 0.0939, 0.2139, 0.2016, 0.2107,
        0.0682, 0.1438], device='cuda:0', grad_fn=<PowBackward0>)
[29/32] Loss_D: 0.6359 Loss_G: 0.5750 D(x): 0.0067 D(G(z)): 0.0084 / 0.0090
tensor([0.2080, 0.0825, 0.0913, 0.1974, 0.2056, 0.2449, 0.1225, 0.1497, 0.2380,
        0.2679, 0.1762, 0.2547, 0.1850, 0.1053, 0.2241, 0.1602, 0.0974, 0.1460,
        0.2275, 0.0602], device='cuda:0', grad_fn=<PowBackward0>)
[30/32] Loss_D: 0.5995 Loss_G: 0.5334 D(x): 0.0084 D(G(z)): 0.0106 / 0.0108
tensor([0.0913, 0.1997, 0.1576, 0.2334, 0.2395, 0.2362, 0.1714, 0.2755, 0.2030,
        0.2114

[25/32] Loss_D: 0.5732 Loss_G: 0.3145 D(x): 0.0093 D(G(z)): 0.0185 / 0.0183
[26/32] Loss_D: 0.5936 Loss_G: 0.3298 D(x): 0.0095 D(G(z)): 0.0182 / 0.0171
[27/32] Loss_D: 0.5779 Loss_G: 0.3362 D(x): 0.0097 D(G(z)): 0.0170 / 0.0170
[28/32] Loss_D: 0.4964 Loss_G: 0.3306 D(x): 0.0136 D(G(z)): 0.0168 / 0.0166
[29/32] Loss_D: 0.5840 Loss_G: 0.3370 D(x): 0.0092 D(G(z)): 0.0168 / 0.0166
[30/32] Loss_D: 0.5456 Loss_G: 0.3407 D(x): 0.0108 D(G(z)): 0.0167 / 0.0165
[31/32] Loss_D: 0.4699 Loss_G: 0.3309 D(x): 0.0143 D(G(z)): 0.0168 / 0.0170
[32/32] Loss_D: 0.3741 Loss_G: 0.3183 D(x): 0.0184 D(G(z)): 0.0175 / 0.0183
-----THE [4/500] epoch end-----
-----THE [5/500] epoch start-----
[1/32] Loss_D: 0.5401 Loss_G: 0.3218 D(x): 0.0117 D(G(z)): 0.0188 / 0.0180
[2/32] Loss_D: 0.5277 Loss_G: 0.3140 D(x): 0.0122 D(G(z)): 0.0193 / 0.0192
[3/32] Loss_D: 0.5075 Loss_G: 0.3062 D(x): 0.0130 D(G(z)): 0.0206 / 0.0201
[4/32] Loss_D: 0.5308 Loss_G: 0.2965 D(x): 0.0127 D(G(z)): 0.0225 / 0.0218
[5/32] Loss_D: 0.4900 Loss

[17/32] Loss_D: 0.5502 Loss_G: 0.2804 D(x): 0.0107 D(G(z)): 0.0223 / 0.0222
tensor([0.3786, 0.3698, 0.3657, 0.3680, 0.3653, 0.3773, 0.3720, 0.3807, 0.3767,
        0.3681, 0.3623, 0.3595, 0.3752, 0.3719, 0.3786, 0.3685, 0.3679, 0.3666,
        0.3625, 0.3684], device='cuda:0', grad_fn=<PowBackward0>)
[18/32] Loss_D: 0.5081 Loss_G: 0.2807 D(x): 0.0129 D(G(z)): 0.0220 / 0.0219
tensor([0.3739, 0.3667, 0.3692, 0.3808, 0.3702, 0.3727, 0.3836, 0.3738, 0.3780,
        0.3692, 0.3761, 0.3818, 0.3802, 0.3819, 0.3772, 0.3799, 0.3686, 0.3717,
        0.3624, 0.3767], device='cuda:0', grad_fn=<PowBackward0>)
[19/32] Loss_D: 0.4948 Loss_G: 0.2759 D(x): 0.0133 D(G(z)): 0.0220 / 0.0223
tensor([0.3831, 0.3691, 0.3828, 0.3781, 0.3768, 0.3842, 0.3685, 0.3812, 0.3808,
        0.3787, 0.3798, 0.3736, 0.3767, 0.3822, 0.3786, 0.3730, 0.3642, 0.3740,
        0.3783, 0.3797], device='cuda:0', grad_fn=<PowBackward0>)
[20/32] Loss_D: 0.5082 Loss_G: 0.2734 D(x): 0.0124 D(G(z)): 0.0222 / 0.0223
tensor([0.3769, 0.

[17/32] Loss_D: 0.4910 Loss_G: 0.2519 D(x): 0.0134 D(G(z)): 0.0245 / 0.0245
[18/32] Loss_D: 0.4595 Loss_G: 0.2660 D(x): 0.0157 D(G(z)): 0.0245 / 0.0234
[19/32] Loss_D: 0.5314 Loss_G: 0.2621 D(x): 0.0116 D(G(z)): 0.0236 / 0.0234
[20/32] Loss_D: 0.5378 Loss_G: 0.2643 D(x): 0.0123 D(G(z)): 0.0236 / 0.0242
[21/32] Loss_D: 0.5405 Loss_G: 0.2694 D(x): 0.0115 D(G(z)): 0.0241 / 0.0239
[22/32] Loss_D: 0.5039 Loss_G: 0.2657 D(x): 0.0126 D(G(z)): 0.0241 / 0.0239
[23/32] Loss_D: 0.5542 Loss_G: 0.2728 D(x): 0.0107 D(G(z)): 0.0237 / 0.0228
[24/32] Loss_D: 0.4562 Loss_G: 0.2701 D(x): 0.0146 D(G(z)): 0.0229 / 0.0231
[25/32] Loss_D: 0.5271 Loss_G: 0.2718 D(x): 0.0120 D(G(z)): 0.0231 / 0.0232
[26/32] Loss_D: 0.5101 Loss_G: 0.2669 D(x): 0.0130 D(G(z)): 0.0229 / 0.0232
[27/32] Loss_D: 0.5782 Loss_G: 0.2789 D(x): 0.0098 D(G(z)): 0.0235 / 0.0228
[28/32] Loss_D: 0.4998 Loss_G: 0.2784 D(x): 0.0124 D(G(z)): 0.0230 / 0.0228
[29/32] Loss_D: 0.5269 Loss_G: 0.2711 D(x): 0.0128 D(G(z)): 0.0229 / 0.0231
[30/32] Loss

...

[28/32] Loss_D: 0.5143 Loss_G: 0.1688 D(x): 0.0135 D(G(z)): 0.0257 / 0.0252
[29/32] Loss_D: 0.4162 Loss_G: 0.1714 D(x): 0.0163 D(G(z)): 0.0250 / 0.0251
[30/32] Loss_D: 0.3962 Loss_G: 0.1666 D(x): 0.0182 D(G(z)): 0.0257 / 0.0258
[31/32] Loss_D: 0.4130 Loss_G: 0.1620 D(x): 0.0179 D(G(z)): 0.0260 / 0.0262
[32/32] Loss_D: 0.4279 Loss_G: 0.1718 D(x): 0.0158 D(G(z)): 0.0255 / 0.0252
-----THE [95/500] epoch end-----
-----THE [96/500] epoch start-----
tensor([0.4917, 0.4920, 0.4884, 0.4860, 0.5032, 0.4779, 0.4696, 0.4961, 0.4673,
        0.4942, 0.4874, 0.4922, 0.4750, 0.4850, 0.4852, 0.4824, 0.4840, 0.4981,
        0.4952, 0.4908], device='cuda:0', grad_fn=<PowBackward0>)
[1/32] Loss_D: 0.4164 Loss_G: 0.1706 D(x): 0.0168 D(G(z)): 0.0254 / 0.0252
tensor([0.4520, 0.4554, 0.4899, 0.4978, 0.4694, 0.4970, 0.4498, 0.4837, 0.5037,
        0.4793, 0.4761, 0.4657, 0.4433, 0.4575, 0.4974, 0.4458, 0.4485, 0.4757,
        0.4457, 0.4812], device='cuda:0', grad_fn=<PowBackward0>)
[2/32] Loss_D: 0.4198 Los

[26/32] Loss_D: 0.4432 Loss_G: 0.1734 D(x): 0.0153 D(G(z)): 0.0261 / 0.0256
tensor([0.4855, 0.4956, 0.5061, 0.4835, 0.4946, 0.4805, 0.4789, 0.4868, 0.5153,
        0.5012, 0.4683, 0.4865, 0.5013, 0.4781, 0.4956, 0.4828, 0.4876, 0.4917,
        0.4897, 0.5043], device='cuda:0', grad_fn=<PowBackward0>)
[27/32] Loss_D: 0.3262 Loss_G: 0.1677 D(x): 0.0211 D(G(z)): 0.0243 / 0.0241
tensor([0.4889, 0.4950, 0.4918, 0.4929, 0.4888, 0.4864, 0.4835, 0.4908, 0.4939,
        0.5022, 0.4848, 0.4952, 0.4871, 0.4977, 0.4973, 0.4848, 0.5150, 0.4991,
        0.4900, 0.5051], device='cuda:0', grad_fn=<PowBackward0>)
[28/32] Loss_D: 0.4685 Loss_G: 0.1653 D(x): 0.0145 D(G(z)): 0.0251 / 0.0259
tensor([0.4877, 0.4871, 0.4796, 0.5033, 0.4895, 0.4876, 0.4817, 0.4949, 0.4903,
        0.4905, 0.5048, 0.5031, 0.4839, 0.4918, 0.4877, 0.4978, 0.4918, 0.5015,
        0.4800, 0.4962], device='cuda:0', grad_fn=<PowBackward0>)
[29/32] Loss_D: 0.3960 Loss_G: 0.1669 D(x): 0.0179 D(G(z)): 0.0245 / 0.0246
tensor([0.4946, 0.

[20/32] Loss_D: 0.4469 Loss_G: 0.1685 D(x): 0.0163 D(G(z)): 0.0248 / 0.0249
[21/32] Loss_D: 0.3990 Loss_G: 0.1673 D(x): 0.0192 D(G(z)): 0.0251 / 0.0248
[22/32] Loss_D: 0.3892 Loss_G: 0.1628 D(x): 0.0186 D(G(z)): 0.0253 / 0.0259
[23/32] Loss_D: 0.4786 Loss_G: 0.1751 D(x): 0.0150 D(G(z)): 0.0257 / 0.0250
[24/32] Loss_D: 0.4251 Loss_G: 0.1772 D(x): 0.0160 D(G(z)): 0.0253 / 0.0245
[25/32] Loss_D: 0.4754 Loss_G: 0.1724 D(x): 0.0139 D(G(z)): 0.0244 / 0.0245
[26/32] Loss_D: 0.4122 Loss_G: 0.1662 D(x): 0.0176 D(G(z)): 0.0250 / 0.0253
[27/32] Loss_D: 0.4557 Loss_G: 0.1747 D(x): 0.0156 D(G(z)): 0.0254 / 0.0250
[28/32] Loss_D: 0.4362 Loss_G: 0.1690 D(x): 0.0155 D(G(z)): 0.0258 / 0.0255
[29/32] Loss_D: 0.4377 Loss_G: 0.1696 D(x): 0.0151 D(G(z)): 0.0242 / 0.0241
[30/32] Loss_D: 0.3782 Loss_G: 0.1687 D(x): 0.0180 D(G(z)): 0.0252 / 0.0253
[31/32] Loss_D: 0.3856 Loss_G: 0.1664 D(x): 0.0177 D(G(z)): 0.0253 / 0.0251
[32/32] Loss_D: 0.2960 Loss_G: 0.1650 D(x): 0.0219 D(G(z)): 0.0245 / 0.0240
-----THE [99

[16/32] Loss_D: 0.4299 Loss_G: 0.1731 D(x): 0.0169 D(G(z)): 0.0245 / 0.0247
tensor([0.4927, 0.4880, 0.4871, 0.4969, 0.4935, 0.4831, 0.4969, 0.4909, 0.4989,
        0.5021, 0.4983, 0.5027, 0.4824, 0.4909, 0.4827, 0.4967, 0.4853, 0.4904,
        0.4837, 0.4952], device='cuda:0', grad_fn=<PowBackward0>)
[17/32] Loss_D: 0.3840 Loss_G: 0.1666 D(x): 0.0181 D(G(z)): 0.0247 / 0.0248
tensor([0.4985, 0.4887, 0.4962, 0.4855, 0.4925, 0.4938, 0.4687, 0.4882, 0.4688,
        0.4912, 0.4916, 0.5025, 0.4920, 0.4906, 0.4966, 0.4877, 0.5005, 0.4937,
        0.4897, 0.4830], device='cuda:0', grad_fn=<PowBackward0>)
[18/32] Loss_D: 0.3734 Loss_G: 0.1682 D(x): 0.0180 D(G(z)): 0.0255 / 0.0251
tensor([0.4835, 0.4837, 0.4876, 0.4687, 0.4748, 0.4892, 0.4785, 0.4921, 0.4974,
        0.4900, 0.4888, 0.4869, 0.4687, 0.4767, 0.4916, 0.4849, 0.4769, 0.4922,
        0.4961, 0.4945], device='cuda:0', grad_fn=<PowBackward0>)
[19/32] Loss_D: 0.5123 Loss_G: 0.1722 D(x): 0.0131 D(G(z)): 0.0251 / 0.0248
tensor([0.4832, 0.

KeyboardInterrupt: 